In [1]:
print("Importing all Libraries")

import os
import gc
import time
import json
import numpy as np
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
from datetime import timedelta, date
import seaborn as sns
import torch
from fastai.tabular.all import *
import pickle
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


Importing all Libraries


In [2]:
!pip install tensorflow

In [3]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from keras.preprocessing.text import one_hot
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Embedding

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format='{:,.4f}'.format
pd.options.display.max_rows=200
pd.set_option('display.max_rows',800)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [5]:
df1 = pd.read_parquet('/content/drive/MyDrive/Dresses_2022_23.parquet', engine='pyarrow')

In [6]:
len(df1)

2831643

In [7]:
df1 = df1[df1['product_gender']=='Female']


In [ ]:
df1['return_reason'].value_counts()

Others            2189659
Size Too Large     235535
Size Too Small     219596
Name: return_reason, dtype: int64

In [ ]:
df1.head()

,id_customer,customer_age,gender,venture_code,order_nr,id_sales_order_item,order_created_at,sku_config,sku_simple,sub_category_type,customer_type,size_brand,size_name,size_system_name,brand_name,product_gender,department_code,color_family,color,buying_planning_cat_type,product_name,image_url,material,reason_type,return_reason,outcome,size_2,size_sys_2,measurements,bust,waist,modelbodymeasurements,modeiswearing,chest,hip
0,"3,440,159.0000",30.0000,female,ph,216469169,34696129,2022-05-13 17:55:31,9F5D1AAFE63E87GS,9F5D1AAFE63E87GS-073CZU,Dresses,Existing Customer,,S/M,International,Miss M,Female,MP,white,"Polka, Off White",Wapp,Polka Dots Button-up French Wrap Dress,https://zstaticph01-a.akamaihd.net/p/miss-m-0113-3806322-1.jpg,,None,Others,net,S/M,International,"One size, Free size /> Bust: 90cm /> Waist: 82cm (wrap waist, adjustable) /> Hips: 104cm /> Length: 102cm /> Please allow 13cm error for manual measurement.",,,,,,
1,"16,626.0000",37.0000,female,sg,268373396,33375701,2022-07-07 23:55:08,F738BAAACFBE0EGS,F738BAAACFBE0EGS-050BEN,Dresses,Existing Customer,,XS,International,ZALORA WORK,Female,RPL,green,Olive Green,Wapp,Button Down Shirt Dress,https://zstaticmy01-a.akamaihd.net/p/zalora-work-3862-5636372-1.jpg,100%Polyester Woven,returned,Size Too Large,returned,XS,International,,,,"Height: 5'9\""\"">Bust: 33.5\""\"">Waist: 25\""\"">Hips: 34.5\""\""",S,,
2,"16,626.0000",37.0000,female,sg,268373396,33375701,2022-07-07 23:55:08,F738BAAACFBE0EGS,F738BAAACFBE0EGS-050BEN,Dresses,Existing Customer,,XS,International,ZALORA WORK,Female,RPL,green,Olive Green,Wapp,Button Down Shirt Dress,https://zstaticmy01-a.akamaihd.net/p/zalora-work-3862-5636372-1.jpg,100%Polyester Woven,returned,Size Too Large,returned,XS,International,,,,,,,
3,"727,440.0000",NaN,female,sg,261797586,34934319,2022-10-28 00:33:02,966DCAA892C0F2GS,966DCAA892C0F2GS-020WTN,Dresses,Existing Customer,,S,International,ZALORA WORK,Female,RPL,pink,Blush,Wapp,Asymmetric Hem Wrap Dress,https://zstaticmy01-a.akamaihd.net/p/zalora-work-5050-6001342-1.jpg,100% Polyester,returned,Size Too Small,returned,S,International,,,,,,,
4,"2,305,452.0000",26.0000,female,sg,247986496,31518705,2022-03-03 21:55:34,874A3AA6E38B7DGS,874A3AA6E38B7DGS-002MSR,Dresses,Existing Customer,,M,International,ZALORA BASICS,Female,RPL,multi,Sage/Black Spots,Wapp,Drawstring Waist Balloon Sleeve Dress,https://zstaticmy01-a.akamaihd.net/p/zalora-basics-6861-0389642-1.jpg,100% Polyester Woven,None,Others,net,M,International,,,,,,,


In [ ]:
len(df1[(df1['bust'].notnull()) & (df1['waist'].notnull()) & (df1['chest'].notnull()) & (df1['hip'].notnull())])

2644790

In [11]:
mask1 = (df1['bust'].notnull()) & (df1['waist'].notnull()) & (df1['hip'].notnull()) & (df1['bust']!= '') & (df1['waist']!= '') & (df1['hip']!='')

df1 = df1[mask1].copy()

In [9]:
len(df1)

328891

In [ ]:
df1['sku_simple'].nunique()

13572

In [ ]:
df1['id_customer'].nunique()

60749

In [ ]:
df1['id_customer'].dtypes

dtype('int64')

In [12]:
df1['id_customer'] = df1['id_customer'].fillna(0).astype('int')
df1['id_customer'].head()

12    1326209
29    4139843
50    1326209
51    1326209
60    4443492
Name: id_customer, dtype: int64

In [ ]:
df1['return_reason'].value_counts()

Others            269641
Size Too Small     30705
Size Too Large     28545
Name: return_reason, dtype: int64

In [13]:
df1['record_keep'] = np.where(df1['outcome']=='net', 1,
                              np.where((df1['outcome']=='returned') & (df1['return_reason'].isin(['Size Too Small', 'Size Too Large'])),1,0))

In [14]:
print(len(df1))
df1 = df1[df1['record_keep']==1]
print(len(df1))

328891
250620


In [15]:
df1['target_size_actual'] = np.where(df1['outcome']=='net', 'Fit',
                                     np.where((df1['outcome']=='returned') & (df1['return_reason']=='Size Too Small'), 'Small',
                                              np.where((df1['outcome']=='returned') & (df1['return_reason']=='Size Too Large'), 'Large', '')))

In [16]:
df1['target_size_actual'].value_counts()

Fit      191482
Small     30628
Large     28510
Name: target_size_actual, dtype: int64

In [17]:
df1['target_size_actual'] = df1['target_size_actual'].str.lower()

In [18]:
df1['target_size_actual'].value_counts()

fit      191482
small     30628
large     28510
Name: target_size_actual, dtype: int64

In [19]:
cols = ['id_customer', 'customer_age', 'gender', 'venture_code',  'sku_config', 'sku_simple',
        'sub_category_type', 'size_brand', 'size_name', 'size_system_name',
        'brand_name', 'product_gender', 'department_code', 'product_name',
        'material', 'bust', 'waist', 'hip', 'target_size_actual']

df2 = df1[cols].drop_duplicates()

In [20]:
#df2.head().to_csv('Purchase_data.csv', index=None)

In [21]:
len(df2[df2['customer_age'].notnull()])/len(df2), len(df2[df2['gender'].notnull()])/len(df2)

(0.6172989236277128, 1.0)

In [22]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df2['size_le'] = le.fit_transform(df2['size_name'])

In [23]:
df2['sku_le'] = le.fit_transform(df2['sku_config'])

In [24]:
df2['sku_le'].dtypes

dtype('int64')

In [25]:
random.seed(1)

# Shuffle dataframe
df2 = df2.sample(frac = 1)

train_data = df2[:int(0.8*len(df2))]
val_data = df2[int(0.8*len(df2)):int(0.9*len(df2))]
test_data = df2[int(0.9*len(df2)):]

len(train_data), len(val_data), len(test_data)

(68600, 8575, 8576)

In [26]:
train_data.dtypes

id_customer             int64
customer_age          float64
gender                 object
venture_code           object
sku_config             object
sku_simple             object
sub_category_type      object
size_brand             object
size_name              object
size_system_name       object
brand_name             object
product_gender         object
department_code        object
product_name           object
material               object
bust                   object
waist                  object
hip                    object
target_size_actual     object
size_le                 int64
sku_le                  int64
dtype: object

In [27]:
train_data.head(1)

,id_customer,customer_age,gender,venture_code,sku_config,sku_simple,sub_category_type,size_brand,size_name,size_system_name,brand_name,product_gender,department_code,product_name,material,bust,waist,hip,target_size_actual,size_le,sku_le
517161,1060264,38.0000,female,hk,7917EAA3CD3B90GS,7917EAA3CD3B90GS-041YMO,Dresses,,M,International,Mango,Female,RBC,Printed Strap Dress,100% Viscose,40,38,48,fit,45,2571


In [28]:
for i,r in train_data[0:10].iterrows():
  print(r['sku_le'])

2571
4190
1694
4919
1831
2436
3189
4944
3808
5166


## Index user and **items**

In [29]:

item_data = {}
item_index = {}
user_index = {}
user_data = {}
u_index = 0
i_index = 0
for index, r in train_data.iterrows():
    if str(r['sku_le']) + '|' + str(r['size_le']) not in item_data:
        item_data[str(r['sku_le']) + '|' + str(r['size_le'])] = [r]
        item_index[str(r['sku_le']) + '|' + str(r['size_le'])] = i_index
        i_index += 1
    else:
        item_data[str(r['sku_le']) + '|' + str(r['size_le'])].append(r)

    if r['id_customer'] not in user_data:
        user_data[r['id_customer']] = [r]
        user_index[r['id_customer']] = u_index
        u_index += 1
    else:
        user_data[r['id_customer']].append(r)

In [30]:
len(user_data), len(user_index), len(item_data), len(item_index)

(44999, 44999, 12799, 12799)

In [31]:
#item_data

In [32]:
true_size_item = np.zeros(len(item_data))
true_size_cust = np.zeros(len(user_data))
w = 1; b_1 = -1; b_2 = 1; lamda = 2

for item in item_data:
    true_size_item[item_index[item]] = int(item.split('|')[1])

In [33]:
true_size_item[:10]

array([45., 19., 57., 33., 57., 51., 57., 44., 21., 52.])

In [34]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [41]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


def calc_auc():
    train_features = []
    train_labels = []
    for i,r in train_data.iterrows():
        fe = []
        fe.append(true_size_cust[user_index[r['id_customer']]])
        fe.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]])
        train_features.append(fe)

        if 'small' in r['target_size_actual']:
            train_labels.append(0)
        elif 'fit' in r['target_size_actual']:
            train_labels.append(1)
        elif 'large' in r['target_size_actual']:
            train_labels.append(2)

    c = 1
    #clf_1LV = LogisticRegression(fit_intercept=True, multi_class='ovr', C=c)



    clf_1LV = XGBClassifier(random_state=1, n_estimators=500, nfold=5, metrics=["mlogloss", "auc"])


    clf_1LV.fit(train_features, train_labels)

    test_features = []; test_labels = []; test_labels_auc = []
    for i,r in test_data.iterrows():
        fe = []
        try:
            fe.append(true_size_cust[user_index[r['id_customer']]])
        except KeyError:
            fe.append(np.mean(true_size_cust))
        try:
            fe.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]])
        except KeyError:
            fe.append(np.mean(true_size_item))

        test_features.append(fe)
        label = [0, 0, 0]
        if 'small' in r['target_size_actual']:
            test_labels.append(0)
            label[0] = 1
        elif 'fit' in r['target_size_actual']:
            test_labels.append(1)
            label[1] = 1
        elif 'large' in r['target_size_actual']:
            test_labels.append(2)
            label[2] = 1
        test_labels_auc.append(label)

    test_labels_auc = np.array(test_labels_auc)

    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import label_binarize

    pred = clf_1LV.predict_proba(test_features)
    AUC = []
    for i in range(3):
        AUC.append(roc_auc_score(test_labels_auc[:,i], pred[:,i], average='weighted'))
    print('Average AUC', np.mean(AUC), AUC)

In [36]:
train_data.head()

,id_customer,customer_age,gender,venture_code,sku_config,sku_simple,sub_category_type,size_brand,size_name,size_system_name,brand_name,product_gender,department_code,product_name,material,bust,waist,hip,target_size_actual,size_le,sku_le
517161,1060264,38.0000,female,hk,7917EAA3CD3B90GS,7917EAA3CD3B90GS-041YMO,Dresses,,M,International,Mango,Female,RBC,Printed Strap Dress,100% Viscose,40,38,48,fit,45,2571
960492,469173,NaN,female,hk,C4E20AAD3C3F3CGS,C4E20AAD3C3F3CGS-063CBC,Dresses,,36,EU,Trendyol,Female,RBO,Side Ruched Mini Dress,"96% Polyester, 4% Elastane",38,31,36,large,19,4190
1185818,8329650,NaN,,id,50E5DAA184ED97GS,50E5DAA184ED97GS-093YPY,Dresses,,XS,International,Something Borrowed,Female,RPL,Scoop Back Pleated Chiffon Swing Dress,Polyblend,36.5,28.5,48.5,fit,57,1694
1836322,1982848,35.0000,female,my,E880AAAF09550BGS,E880AAAF09550BGS-034KGB,Dresses,,46,EU,Trendyol,Female,RBO,Plus Size Floral Print Dress,100% Viscose,61,51,66.5,fit,33,4919
360060,8259537,25.0000,female,id,56A98AACD78EF0GS,56A98AACD78EF0GS-046CEJ,Dresses,,XS,International,ZALORA,Female,RPL,V-Neck Cold Shoulder Dress,100% Polyester,38.5,34.5,45.5,fit,57,1831


In [42]:
def f(s,t):
    return w*(s-t)

def cal_loss_user(user, cust_size):
    loss = 0
    for r in user_data[user]:
        if 'small' in r['target_size_actual']:
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]) + b_2)
        elif 'fit' in r['target_size_actual']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]) - b_2)
            loss += max(0, 1 - f(cust_size, true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]) + b_1)
        elif 'large' in r['target_size_actual']:
            loss += max(0, 1 + f(cust_size, true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]) - b_1)
    return loss

In [43]:
def cal_loss_item(item, product_size):
    loss = 0
    for r in item_data[item]:
        if 'small' in r['target_size_actual']:
            loss += max(0, 1 - f(true_size_cust[user_index[r['id_customer']]], product_size) + b_2)
        elif 'fit' in r['target_size_actual']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['id_customer']]], product_size) - b_2)
            loss += max(0, 1 - f(true_size_cust[user_index[r['id_customer']]], product_size) + b_1)
        elif 'large' in r['target_size_actual']:
            loss += max(0, 1 + f(true_size_cust[user_index[r['id_customer']]], product_size) - b_1)
    return loss

In [44]:
def total_loss():
    loss = 0
    for item in item_data:
        for r in item_data[item]:
            product_size = true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]
            if 'small' in r['target_size_actual']:
                loss += max(0, 1 - f(true_size_cust[user_index[r['id_customer']]], product_size) + b_2)
            elif 'fit' in r['target_size_actual']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['id_customer']]], product_size) - b_2)
                loss += max(0, 1 - f(true_size_cust[user_index[r['id_customer']]], product_size) + b_1)
            elif 'large' in r['target_size_actual']:
                loss += max(0, 1 + f(true_size_cust[user_index[r['id_customer']]], product_size) - b_1)
    return loss

In [46]:
for iterr in range(0,20):

    ## Phase 1
    for user in user_data:
        candidate_sizes = []
        for r in user_data[user]:
            if 'small' in r['target_size_actual']:
                candidate_sizes.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]] + ((b_2+1)/w))
            elif 'fit' in r['target_size_actual']:
                candidate_sizes.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]] + ((b_1+1)/w))
                candidate_sizes.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]] + ((b_2-1)/w))
            elif 'large' in r['target_size_actual']:
                candidate_sizes.append(true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]] + ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)

        if len(candidate_sizes) == 1:
            true_size_cust[user_index[user]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_user(user, candidate_sizes[s]) - cal_loss_user(user, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_cust[user_index[user]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_cust[user_index[user]] = candidate_sizes[-1]

    ## Phase 2
    for item in item_data:
        candidate_sizes = []
        for r in item_data[item]:
            if 'small' in r['target_size_actual']:
                candidate_sizes.append(true_size_cust[user_index[r['id_customer']]] - ((b_2+1)/w))
            elif 'fit' in r['target_size_actual']:
                candidate_sizes.append(true_size_cust[user_index[r['id_customer']]] - ((b_1+1)/w))
                candidate_sizes.append(true_size_cust[user_index[r['id_customer']]] - ((b_2-1)/w))
            elif 'large' in r['target_size_actual']:
                candidate_sizes.append(true_size_cust[user_index[r['id_customer']]] - ((b_1-1)/w))

        flag = 0
        candidate_sizes = list(set(candidate_sizes))
        candidate_sizes = sorted(candidate_sizes)
        if len(candidate_sizes) == 1:
            true_size_item[item_index[item]] = candidate_sizes[0]
        else:
            for s in range(1, len(candidate_sizes)):
                slope = (cal_loss_item(item, candidate_sizes[s]) - cal_loss_item(item, candidate_sizes[s-1]))/(candidate_sizes[s] - candidate_sizes[s-1])
                if slope>=0:
                    flag=1
                    true_size_item[item_index[item]] = candidate_sizes[s-1]
                    break

            if flag==0:
                true_size_item[item_index[item]] = candidate_sizes[-1]

    ## Phase 3
    learning_rate = 0.0000005/np.sqrt(iterr+1)
    grad_w = 0
    grad_b1 = 0
    grad_b2 = 0
    for i,r in train_data.iterrows():
        s = true_size_cust[user_index[r['id_customer']]]
        t = true_size_item[item_index[str(r['sku_le']) + '|' + str(r['size_le'])]]

        if 'small' in r['target_size_actual']:
            A = 1 - f(s, t) + b_2
            if A>0:
                grad_w += -1*(s - t)
                grad_b2 += 1
        elif 'fit' in r['target_size_actual']:
            B = 1 + f(s, t) - b_2
            C = 1 - f(s, t) + b_1
            if B>0:
                grad_w += (s - t)
                grad_b2 += -1
            if C>0:
                grad_w += -1*(s - t)
                grad_b1 += 1
        elif 'large' in r['target_size_actual']:
            D = 1 + f(s, t) - b_1
            if D>0:
                grad_w += (s - t)
                grad_b1 += -1

    w -= learning_rate*(grad_w + 2*lamda*w)
    b_1 -= learning_rate*(grad_b1 + 2*lamda*b_1)
    b_2 -= learning_rate*(grad_b2 + 2*lamda*b_2)
    if iterr%5 == 0:
        print(iterr, total_loss())
        calc_auc()

0 75757.51814349667
[14:15:38] WARNING: ../src/learner.cc:767: 
Parameters: { "metrics", "nfold" } are not used.

Average AUC 0.6176871770896125 [0.6523552567764219, 0.6001627678964653, 0.6005435065959506]
5 62588.85142085235
[14:18:09] WARNING: ../src/learner.cc:767: 
Parameters: { "metrics", "nfold" } are not used.

Average AUC 0.6411538203787147 [0.6520677532615615, 0.6499266530953202, 0.6214670547792627]
10 54288.36885730359
[14:20:42] WARNING: ../src/learner.cc:767: 
Parameters: { "metrics", "nfold" } are not used.

Average AUC 0.627794733907655 [0.6488581021573536, 0.6260682648303463, 0.6084578347352648]
15 48069.339222314826
[14:23:15] WARNING: ../src/learner.cc:767: 
Parameters: { "metrics", "nfold" } are not used.

Average AUC 0.6123581920889011 [0.646299605654632, 0.6013070274801815, 0.5894679431318899]
